# SVM với scikit-learn

Mục tiêu của notebook này là xây dựng mô hình SVM sử dụng thư viện scikit-learn để dự đoán cảm xúc từ các bài bình luận sản phẩm. Chúng ta sẽ thực hiện:

 * Load tập dữ liệu các bài bình luận như notebook trước.
 * Triển khai mô hình SVM sử dụng scikit-learn.
 * Điều chỉnh một vài tham số.

In [1]:
import pandas
import numpy as np
from sklearn.model_selection import train_test_split
import json

def get_numpy_data(dataframe, features, label):
    dataframe.loc[:, 'intercept'] = 1
    features = ['intercept'] + features
    feature_matrix = dataframe.loc[:, features].values
    label_array = dataframe.loc[:, label].values
    return (feature_matrix, label_array)

def remove_punctuation(text):
    import string
    return text.translate(string.punctuation)

def get_product_reviews_data():
    products_df = pandas.read_csv('amazon_baby_subset.csv')

    with open('important_words.json', 'r') as f:
        important_words = json.loads(f.read())

    products_df = products_df.fillna({'review':''})  # fill in N/A's in the review column
    products_df.loc[:, 'review_clean'] = products_df['review'].apply(remove_punctuation)

    for word in important_words:
        products_df.loc[:, word] = products_df['review_clean'].apply(lambda s : s.split().count(word))

    sentiment_train_data = products_df.sample(frac=0.8, random_state=100)
    sentiment_validation_data = products_df.drop(sentiment_train_data.index)

    sentiment_X_train, sentiment_y_train = get_numpy_data(sentiment_train_data, important_words, 'sentiment')
    sentiment_X_valid, sentiment_y_valid = get_numpy_data(sentiment_validation_data, important_words, 'sentiment')

    print ('*****Sentiment data shape*****')
    print ('sentiment_X_train.shape: ', sentiment_X_train.shape)
    print ('sentiment_y_train.shape: ', sentiment_y_train.shape)
    print ('sentiment_X_valid.shape: ', sentiment_X_valid.shape)
    print ('sentiment_y_valid.shape: ', sentiment_y_valid.shape)

    return (sentiment_X_train, sentiment_y_train), (sentiment_X_valid, sentiment_y_valid)

In [2]:
# Import một số thư viện
import pandas
import numpy as np

## Load tập dữ liệu product reviews
Như mô-đun trước, chúng ta sẽ load, tiền xử lý dữ liệu, chuyển đổi và chia chúng thành các tập huấn luyện và kiểm tra. Trong notebook này, chúng ta không tập trung vào điều đó nên chỉ cần chạy các cell sau. Có thể kiểm tra code dữ liệu bên trong folder **utils**.

In [3]:
train_set, val_set = get_product_reviews_data()

sentiment_X_train, sentiment_y_train = train_set
sentiment_X_valid, sentiment_y_valid = val_set

C:\Users\lengo\miniconda3\envs\Funix\lib\site-packages\pandas\core\indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


*****Sentiment data shape*****
sentiment_X_train.shape:  (42458, 194)
sentiment_y_train.shape:  (42458,)
sentiment_X_valid.shape:  (10614, 194)
sentiment_y_valid.shape:  (10614,)


# Xây dựng phân loại với scikit-learn
Giờ hãy sử dụng SVM có sẵn của sklearn: [sklearn.svm.LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html). Hãy xem thêm tài liệu của sklearn biết thêm chi tiết về lớp LinearSVC. LinearSVC classifier tương đương với SVM có linear kernel nên chúng ta có thể dùng nó vì tập dữ liệu khá lớn. 

In [4]:
from sklearn.svm import LinearSVC
sentiment_clf = LinearSVC(C=1, random_state=0, max_iter=2000)
sentiment_clf.fit(sentiment_X_train, sentiment_y_train)

print ("***Sentiment result***")
print("Train accuracy: {}".format(sentiment_clf.score(sentiment_X_train, sentiment_y_train)))
print("Validation accuracy: {}".format(sentiment_clf.score(sentiment_X_valid, sentiment_y_valid)))

***Sentiment result***
Train accuracy: 0.7775448678694239
Validation accuracy: 0.770680233653665


C:\Users\lengo\miniconda3\envs\Funix\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# Điều chỉnh siêu tham số 
Hãy thử điều chỉnh một vài siêu tham số để xem liệu có được kết quả tốt hơn không. Hãy dùng [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) để điều chỉnh các tham số C trong SVM classifer.

In [5]:
from sklearn.model_selection import GridSearchCV

tuned_parameters = {
    "C": [1, 2, 5, 10, 20, 100]
}
tuned_sentiment_cls = GridSearchCV(
                        LinearSVC(C=1),
                        param_grid=tuned_parameters,
                        n_jobs=2,
                        verbose=1,
)

tuned_sentiment_cls.fit(sentiment_X_train, sentiment_y_train)
print ("***Sentiment result***")
print("Train accuracy: {}".format(tuned_sentiment_cls.score(sentiment_X_train, sentiment_y_train)))
print("Validation accuracy: {}".format(tuned_sentiment_cls.score(sentiment_X_valid, sentiment_y_valid)))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
***Sentiment result***
Train accuracy: 0.7780394743040181
Validation accuracy: 0.771528170341059


C:\Users\lengo\miniconda3\envs\Funix\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Như vậy chúng ta có được kết quả tốt hơn một chút. 
<br>
**Quiz**: Validation accuracy là bao nhiêu?
<br>
**Đáp án**: 77.15